In [1]:
import torch
torch.__version__

'1.8.0'

## Linear Regression

Regression refers to a set of methods for modeling relationship between one or more independent variables and a dependent variable. When the relationship between variables is linear, then it is expressed as a Linear Regression. 

This relationship is expressed as a linear equation, $y = xw + b$, where x is the set of independent variables multiplied by some weights, b is a bias and y is the dependent variable.

## Linear Model

When the inputs consist of $d$ parameters/features/attributes, our prediction $\hat{y}$ is 
$$ \hat{y} = w_1x_1 + ... + w_dx_d + b $$

The above equation can also be expressed as 
$$ \hat{y} = w^Tx+b $$
where $x \in R^{d}$ is a features vector and $w \in R^{d}$ is a weights vector and their dot product will be the weighted sum of the product of feature vectors with the weights vector. 

Normally, the dataset consist of more than a single example, therefore there will be a features matrix not the features vector, where each row represents an observation or sample. 

The above equation will be - $\hat{y} = Xw + b$, where $X \in R^{n*d}$

> Note: Broadcasting is applied during the summation

## Training Dataset

To find the weights and bais of a model for regression, we need a Training dataset consisting of both the independent variables and a dependent variable. 

Given features of a training dataset X and corresponding labels y, the goal is to find $w$ and $b$ such that the model makes predictions with the least error. Therefore, we need to two things to accomplish this task. 
- A quality measure to find the accuracy of the model (hint: norms).
- A procedure for updating the model to improve its quality.

## Loss Function

Loss function is a non-negative function that is used for comparing the ground-truth values to the predicted values. Norms are normally used as loss functions, as they have the same origin and the lower the number they produce the better the predictions will be. 

The most popular function is a squared error, which is a $L_2$ norm:
$$l^{(i)}(w,b) = \frac{1}{2}(\hat{y}^{(i)} - y^{(i)})^2$$

TO measure the quality of the model with n training examples, we calculate the average of all the squared differences, and the loss function can be expressed as:
$$L(w,b) = \frac{1}{n} \sum_{i=1}^nl^{(i)}(w,b) =  \frac{1}{n}\sum_{i=1}^n\frac{1}{2} (w^Tx^{(i)} + n - y^{(i)})^2 $$ 

As training the model is an optimisation problem, we find parameters that minimise the loss function across all training examples: 
$$w^*,b^* = argmin_{w,b} L(w, b) $$

## Gradient Descent (GD)

There is an mathematical equation that can calculate the value of weights and bias depending on the data supplied, however it is too rigid and can only solve problem for a specific equation. In other words, for each model there is an equation to find weights and bias. This approach does not generalise for all the models and it is difficult to derive. 

Gradient Descent is a key technique used for optimising nearly any deep learning model. It is an algorithm that iteratively reduce the loss produced by the error by updating the parameters (weights and bias) in the direction of the minima of the loss function. 

We use partial derivative in this algorithm to calculate the change in loss function w.r.t the parameters:

$$w = w - \frac{\eta}{N} \sum_{i=1}^N \triangledown_wl^{(i)}(w, b)$$
$$b = b - \frac{\eta}{N} \sum_{i=1}^N \frac{\delta l^{(i)}(w, b)}{\delta b}$$

where $\eta$ is a positive scaler called the __Learning Rate__. We randomly initialise the values of the model paramters, and use Gradient Descent to produce the optimal sets of parameters.

## Minibatch Stochastic Gradient Descent
It is a type of gradient descent algorithm that only takes a random minibatch of samples every time it compute the weights. Normal Gradient Descent takes time to compute as it runs over the whole dataset in each training iteration, minibatch solves this problem by using only a small subset at a time, therefore minibatch takes comparatively lower time to find paramteres. The above equations can be updated to:
$$w = w - \frac{\eta}{|\beta|} \sum_{i=1}^{|\beta|} \triangledown_wl^{(i)}(w, b)$$
$$b = b - \frac{\eta}{|\beta|} \sum_{i=1}^{|\beta|} \frac{\delta l^{(i)}(w, b)}{\delta b}$$

In this method, there are two minimum two hyperparamters required: Batchsize and Learning Rate. hyperparamter tuning is a process by which hyperparamters are chosen based on results on a separate __Validation__ set.

## Deep Learning Pipeline

1. Create and read the dataset
2. Prepare the modeling 
3. Initialise the parameters (Weights and Bias)
4. Define the loss function (MSE)
5. Choose the training algorithms (SGD|ADAM)
6. Run the training method

In [21]:
# Generating the Dataset

def prepare_data(w, b, num_examples):
    """ Generate y = Xw + b + noise """
    X = torch.normal(0, 1 ,(num_examples, len(w)))
    y = torch.mm(X, w) + b
    y += torch.normal(0, 0.01, y.size())
    return X, y.reshape(-1, 1)

true_w = torch.tensor([[2], [-3.3]])
true_b = torch.tensor([4.5])
features, labels = prepare_data(true_w, true_b, 1000)

In [22]:
# Reading the dataset

dataset = torch.utils.data.TensorDataset(features, labels)
batch_size = 10
data_iter = torch.utils.data.DataLoader(dataset, batch_size, shuffle=True)

In [25]:
# Define the model

num_of_inp, num_of_out = 2, 1
net = torch.nn.Linear(num_of_inp, num_of_out)

In [26]:
# Initialise the parameters
net.weight.data.normal_(0, 0.01)
net.bias.data.fill_(0)

tensor([0.])

In [27]:
# Choose the loss function
loss = torch.nn.MSELoss()

In [28]:
# Choose the training algorithms (SGD|ADAM)
optimiser = torch.optim.SGD(net.parameters(), lr=0.001)

In [29]:
# Run the training method
num_epochs = 50
for epoch in range(num_epochs):
    for X, y in data_iter:
        y_hat = net(X)
        l = loss(y_hat, y)
        optimiser.zero_grad()
        l.backward()
        optimiser.step()
    l = loss(net(features), labels)
    print(f"epoch {epoch+1}, loss {l:f}")

epoch 1, loss 24.812777
epoch 2, loss 15.937796
epoch 3, loss 10.248366
epoch 4, loss 6.597397
epoch 5, loss 4.251769
epoch 6, loss 2.743270
epoch 7, loss 1.772013
epoch 8, loss 1.145958
epoch 9, loss 0.742014
epoch 10, loss 0.481031
epoch 11, loss 0.312233
epoch 12, loss 0.202919
epoch 13, loss 0.132049
epoch 14, loss 0.086042
epoch 15, loss 0.056144
epoch 16, loss 0.036690
epoch 17, loss 0.024015
epoch 18, loss 0.015748
epoch 19, loss 0.010351
epoch 20, loss 0.006821
epoch 21, loss 0.004510
epoch 22, loss 0.002997
epoch 23, loss 0.002004
epoch 24, loss 0.001352
epoch 25, loss 0.000923
epoch 26, loss 0.000641
epoch 27, loss 0.000455
epoch 28, loss 0.000333
epoch 29, loss 0.000252
epoch 30, loss 0.000198
epoch 31, loss 0.000163
epoch 32, loss 0.000140
epoch 33, loss 0.000124
epoch 34, loss 0.000114
epoch 35, loss 0.000107
epoch 36, loss 0.000103
epoch 37, loss 0.000100
epoch 38, loss 0.000098
epoch 39, loss 0.000096
epoch 40, loss 0.000095
epoch 41, loss 0.000095
epoch 42, loss 0.00009

In [30]:
new_w = net.weight.data
new_b = net.bias.data
print("True weights ({}), Predicted weights ({}), error in estimating w: {}".format(true_w, new_w, true_w - new_w.reshape(true_w.shape)))
print("True bais ({}), Predicted bais ({}), error in estimating b: {}".format(true_b, new_b, true_b - new_b.reshape(true_b.shape)))

True weights (tensor([[ 2.0000],
        [-3.3000]])), Predicted weights (tensor([[ 2.0001, -3.3007]])), error in estimating w: tensor([[-5.7220e-05],
        [ 6.5303e-04]])
True bais (tensor([4.5000])), Predicted bais (tensor([4.4999])), error in estimating b: tensor([0.0001])
